In [1]:
#Libraries
import re
import pandas as pd
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
import seaborn as sns
from googletrans import Translator
from IPython.display import display

# from sklearn.impute import SimpleImputer

import re, nltk

from __future__ import division
from collections import Counter
# import re, nltk


In [232]:
WORDS = nltk.corpus.brown.words()
COUNTS = Counter(WORDS)

In [234]:
def trans(text):

    translator = Translator()

    trans_text = translator.translate(text)

    trans_src = trans_text.src

    return trans_src


def pdist(counter):
    "Make a probability distribution, given evidence from a Counter."
    N = sum(counter.values())
    return lambda x: counter[x]/N

P = pdist(COUNTS)

def Pwords(words):
    "Probability of words, assuming each word is independent of others."
    return product(P(w) for w in words)

def product(nums):
    "Multiply the numbers together.  (Like `sum`, but with multiplication.)"
    result = 1
    for x in nums:
        result *= x
    return result

def splits(text, start=0, L=20):
    "Return a list of all (first, rest) pairs; start <= len(first) <= L."
    return [(text[:i], text[i:]) 
            for i in range(start, min(len(text), L)+1)]

def segment(text):
    "Return a list of words that is the most probable segmentation of text."
    if not text: 
        return []
    else:
        candidates = ([first] + segment(rest) 
                      for (first, rest) in splits(text, 1))
        return max(candidates, key=Pwords)

In [94]:
chat_df = pd.read_csv(r'ChatDetailsDataset\2022_09150915_CD_RD.csv')
# DDA_LC_APP_28 = pd.read_csv('2022_05280528_DDA_LC_APP.csv')

In [95]:
# display(DDA_LC_APP_25.shape)
# # display(DDA_LC_APP_28.shape)
# DDA_chat_df0619.shape

chat_df.shape

(10463, 41)

In [96]:
# len(DDA_0619.columns)

chat_df.head(2)

,Create Date,tenant_name,robot_code,robot_name,from_id,from_name,Session_id,Intention,user_id,Seller id (X-space),...,is_session_direct_liveagent,Contacted live agent on same day,Feedback Card Response,is_chat_recommend,is_chat_no answer,is_chat_pre/post match,live agent_session_id,Last Query Before Live Agent Session,Live Agent User Query,Live Agent Response
0,20220915,Alime-DRZ-CC-BD,l2Cgc9psxK,BD-Daz-EN,V3EasrtOgs,BD-Daz-EN-HC-App,93235e3848fc431185acb1103c1d1a2c,NaN,2101100080989538,7.005152e+11,...,N,N,Not Available Yet,N,N,Y,NaN,I want to know about returns.,NaN,NaN
1,20220915,Alime-DRZ-CC-BD,l2Cgc9psxK,BD-Daz-EN,V3EasrtOgs,BD-Daz-EN-HC-App,93235e3848fc431185acb1103c1d1a2c,NaN,2101100080989538,7.005152e+11,...,N,N,Not Available Yet,N,N,Y,NaN,I want to know about returns.,NaN,NaN


In [97]:
chat_df.tail(2)

,Create Date,tenant_name,robot_code,robot_name,from_id,from_name,Session_id,Intention,user_id,Seller id (X-space),...,is_session_direct_liveagent,Contacted live agent on same day,Feedback Card Response,is_chat_recommend,is_chat_no answer,is_chat_pre/post match,live agent_session_id,Last Query Before Live Agent Session,Live Agent User Query,Live Agent Response
10461,20220915,Alime-DRZ-CC-BD,l2Cgc9psxK,BD-Daz-EN,V3EasrtOgs,BD-Daz-EN-HC-App,e61766ac7d87484d868ca5d1fd00a841,route_daraz_others,2101100082343179,NaN,...,N,N,Not Available Yet,N,N,N,NaN,Why do I see failed delivery attempt?,NaN,NaN
10462,20220915,Alime-DRZ-CC-BD,l2Cgc9psxK,BD-Daz-EN,V3EasrtOgs,BD-Daz-EN-HC-App,e61766ac7d87484d868ca5d1fd00a841,route_daraz_others,2101100082343179,NaN,...,N,N,Not Available Yet,N,N,N,NaN,Why do I see failed delivery attempt?,NaN,NaN


In [98]:
chat_df.columns

Index(['Create Date', 'tenant_name', 'robot_code', 'robot_name', 'from_id',
       'from_name', 'Session_id', 'Intention', 'user_id',
       'Seller id (X-space)', 'user_nick', 'create_time', 'create_time_hour',
       'chat_uuid', 'Enter Type', 'query', 'Question Type', 'sopname',
       'answer_type', 'SOP_Solution_id', 'SOP_Solution_Name',
       'Standard Knowledge Title', 'Knowledge_id', 'knowledge_cate_1_name',
       'knowledge_cate_2_name', 'knowledge_cate_3_name',
       'is_session_unsatisfied', 'is_session_last_no answer',
       'call_agent_by_click_shortcut', 'is_session_last_recommend no click',
       'call_agent_by_intention', 'is_session_direct_liveagent',
       'Contacted live agent on same day', 'Feedback Card Response',
       'is_chat_recommend', 'is_chat_no answer', 'is_chat_pre/post match',
       'live agent_session_id', 'Last Query Before Live Agent Session',
       'Live Agent User Query', 'Live Agent Response'],
      dtype='object')

In [99]:
chat_df['Create Date'].value_counts()

20220915    10463
Name: Create Date, dtype: int64

In [100]:
hours = [0, 1, 2, 3, 4, 5, 6, 7, 8]
hours = []

In [110]:
chat_df_non_working = chat_df[chat_df['create_time_hour'] < 9]

In [102]:
chat_df_non_working['create_time_hour'].value_counts()

0    1254
1     964
2     744
3     326
8     274
4     191
7     136
5      79
6      72
Name: create_time_hour, dtype: int64

In [ ]:
chat_df['live agent_session_id']

In [111]:
chat_df_no_na = chat_df[chat_df['live agent_session_id'].notna()]

In [105]:

working_hours = [9, 10, 11, 12, 13, 14, 15, 16, 17, 18]
non_working_hours = [0, 1, 2, 3, 4, 5, 6, 7, 8]

In [113]:
chat_df_no_na[chat_df_no_na['create_time_hour'].isin(non_working_hours)][['create_time_hour', 'create_time', 'from_name', 'Intention', 'chat_uuid','live agent_session_id','is_session_unsatisfied', 'is_session_last_no answer','call_agent_by_click_shortcut', 'is_session_last_recommend no click', 'call_agent_by_intention', 'is_session_direct_liveagent', 'Contacted live agent on same day', 'is_chat_recommend', 'is_chat_no answer', 'is_chat_pre/post match', 'Feedback Card Response', 'knowledge_cate_1_name', 'Live Agent User Query', 'Enter Type', 'user_nick']].to_clipboard()

In [86]:

chat_uuid_list =  chat_df_non_working_discrepancy_hour['chat_uuid'].to_list()

In [87]:
chat_df.columns

Index(['Create Date', 'tenant_name', 'robot_code', 'robot_name', 'from_id',
       'from_name', 'Session_id', 'Intention', 'user_id',
       'Seller id (X-space)', 'user_nick', 'create_time', 'create_time_hour',
       'chat_uuid', 'Enter Type', 'query', 'Question Type', 'sopname',
       'answer_type', 'SOP_Solution_id', 'SOP_Solution_Name',
       'Standard Knowledge Title', 'Knowledge_id', 'knowledge_cate_1_name',
       'knowledge_cate_2_name', 'knowledge_cate_3_name',
       'is_session_unsatisfied', 'is_session_last_no answer',
       'call_agent_by_click_shortcut', 'is_session_last_recommend no click',
       'call_agent_by_intention', 'is_session_direct_liveagent',
       'Contacted live agent on same day', 'Feedback Card Response',
       'is_chat_recommend', 'is_chat_no answer', 'is_chat_pre/post match',
       'live agent_session_id', 'Last Query Before Live Agent Session',
       'Live Agent User Query', 'Live Agent Response'],
      dtype='object')

In [88]:
chat_df['Intention'].value_counts()

route_daraz_others       12036
route_daraz_liveagent     1755
ordernumber                239
Name: Intention, dtype: int64

In [ ]:
chat_df['create_time_hour'].isin(non_working_hours)

In [107]:
chat_df[chat_df['create_time_hour'].isin(non_working_hours)][['create_time_hour', 'create_time', 'from_name', 'Intention', 'chat_uuid','live agent_session_id','is_session_unsatisfied', 'is_session_last_no answer','call_agent_by_click_shortcut', 'is_session_last_recommend no click', 'call_agent_by_intention', 'is_session_direct_liveagent', 'Contacted live agent on same day', 'is_chat_recommend', 'is_chat_no answer', 'is_chat_pre/post match', 'Feedback Card Response', 'knowledge_cate_1_name', 'Live Agent User Query', 'Enter Type', 'user_nick']]

,create_time_hour,create_time,from_name,Intention,chat_uuid,live agent_session_id,is_session_unsatisfied,is_session_last_no answer,call_agent_by_click_shortcut,is_session_last_recommend no click,...,is_session_direct_liveagent,Contacted live agent on same day,is_chat_recommend,is_chat_no answer,is_chat_pre/post match,Feedback Card Response,knowledge_cate_1_name,Live Agent User Query,Enter Type,user_nick
0,2,2022-09-15 02:12:55,BD-Daz-EN-HC-App,NaN,c14ad1af1181465baa608564b1bfa74e,NaN,N,N,N,N,...,N,N,N,N,Y,Not Available Yet,NaN,NaN,NaN,Mask Sumon
1,2,2022-09-15 02:13:19,BD-Daz-EN-HC-App,NaN,b056855d25634e1c93024db8620ef84f,NaN,N,N,N,N,...,N,N,N,N,Y,Not Available Yet,NaN,NaN,NaN,Mask Sumon
2,2,2022-09-15 02:21:37,BD-Daz-LC-DD-App,NaN,8d7ecc327ebe4beab962d73228a522ad,NaN,N,N,N,N,...,N,N,N,N,Y,Not Available Yet,NaN,NaN,NaN,Tarak Ahmed Chy
3,2,2022-09-15 02:21:44,BD-Daz-LC-DD-App,NaN,e80529cfa70d4b64ab7b681c0cdc502e,NaN,N,N,N,N,...,N,N,N,N,Y,Not Available Yet,NaN,NaN,NaN,Tarak Ahmed Chy
4,2,2022-09-15 02:21:50,BD-Daz-LC-DD-App,NaN,a1a4a37af0af49ad808e8ae6a45d1752,NaN,N,N,N,N,...,N,N,N,N,Y,Not Available Yet,NaN,NaN,NaN,Tarak Ahmed Chy
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10443,2,2022-09-15 02:16:55,BD-Daz-EN-HC-Web,route_daraz_others,0fa14c5c1b684d6c999e9c67f0453b92,NaN,N,N,N,N,...,N,N,N,N,N,Not Available Yet,NaN,NaN,NaN,Nasim
10444,3,2022-09-15 03:52:10,BD-Daz-EN-HC-Web,route_daraz_others,6e908da8a78041218987c526f8a01a9a,NaN,N,N,N,N,...,N,N,N,N,N,Not Available Yet,NaN,NaN,NaN,Rezwan Chowdhury
10445,3,2022-09-15 03:52:32,BD-Daz-EN-HC-Web,route_daraz_others,c3afa92382e04cbaaf67cc0e1658b607,NaN,N,N,N,N,...,N,N,N,N,N,Not Available Yet,NaN,NaN,NaN,Rezwan Chowdhury
10446,3,2022-09-15 03:51:59,BD-Daz-EN-HC-Web,NaN,4f82cfcd87024023a21c32025fedc93c,NaN,N,N,N,N,...,N,N,N,N,Y,Not Available Yet,NaN,NaN,NaN,Rezwan Chowdhury


In [53]:
chat_df_non_working_discrepancy_hour[chat_df_non_working_discrepancy_hour['create_time_hour'].isin(hours)]

,create_time,create_time_hour,chat_uuid,live agent_session_id
3563,2022-09-15 01:34:15,1,4f19cf6ba1a647b7af6104da0a6fbab2,a7b74a25e64a498e80964598da1b35b8
3564,2022-09-15 01:34:23,1,15aed08ee82443309dcdb86ebcdee7af,a7b74a25e64a498e80964598da1b35b8
3565,2022-09-15 01:33:28,1,38fca3d2090b4daa815627b5db7f659b,a7b74a25e64a498e80964598da1b35b8
3566,2022-09-15 01:34:13,1,7e76ec4385fa4e5b88a246fa7663fb8b,a7b74a25e64a498e80964598da1b35b8
3567,2022-09-15 01:33:41,1,1f375061213d47cd95744a368097324e,a7b74a25e64a498e80964598da1b35b8
3568,2022-09-15 01:34:17,1,6719c9e1b69c44c69870e594ca97f879,a7b74a25e64a498e80964598da1b35b8
3569,2022-09-15 01:33:15,1,4b5fefe7787544ce8f33b1e3952ffbc6,a7b74a25e64a498e80964598da1b35b8
3570,2022-09-15 01:34:19,1,f7aae8035a90451baaa7f5a71d35652f,a7b74a25e64a498e80964598da1b35b8
3571,2022-09-15 01:34:20,1,ebfedcb5b77043319a08bd4eecf3593d,a7b74a25e64a498e80964598da1b35b8
4065,2022-09-15 00:18:04,0,b842be328e294adbac6b5abf5ef7ecea,d6cec73c4fe9490698af5d5305507b0c


In [45]:
chat_df_non_working[chat_df_non_working['create_time_hour'] >= 0]

,create_time,create_time_hour,chat_uuid,live agent_session_id
0,2022-09-15 02:12:55,2,c14ad1af1181465baa608564b1bfa74e,NaN
1,2022-09-15 02:13:19,2,b056855d25634e1c93024db8620ef84f,NaN
2,2022-09-15 02:21:37,2,8d7ecc327ebe4beab962d73228a522ad,NaN
3,2022-09-15 02:21:44,2,e80529cfa70d4b64ab7b681c0cdc502e,NaN
4,2022-09-15 02:21:50,2,a1a4a37af0af49ad808e8ae6a45d1752,NaN
...,...,...,...,...
10443,2022-09-15 02:16:55,2,0fa14c5c1b684d6c999e9c67f0453b92,NaN
10444,2022-09-15 03:52:10,3,6e908da8a78041218987c526f8a01a9a,NaN
10445,2022-09-15 03:52:32,3,c3afa92382e04cbaaf67cc0e1658b607,NaN
10446,2022-09-15 03:51:59,3,4f82cfcd87024023a21c32025fedc93c,NaN


In [44]:
chat_df_non_working[chat_df_non_working['live agent_session_id'].notna()].shape

(17, 4)

In [24]:
chat_df[chat_df['create_time_hour'] == 8]

,Create Date,tenant_name,robot_code,robot_name,from_id,from_name,Session_id,Intention,user_id,Seller id (X-space),...,is_session_direct_liveagent,Contacted live agent on same day,Feedback Card Response,is_chat_recommend,is_chat_no answer,is_chat_pre/post match,live agent_session_id,Last Query Before Live Agent Session,Live Agent User Query,Live Agent Response
201,20220914,Alime-DRZ-CC-BD,l2Cgc9psxK,BD-Daz-EN,0zKpjMUW7x,BD-Daz-EN-HC-Web,5053557a03aa42f0baa41d2c19c4f7c9,route_daraz_others,2101100077657298,8.989200e+04,...,N,N,Not Available Yet,N,N,N,NaN,collection point working hour,NaN,NaN
463,20220914,Alime-DRZ-CC-BD,l2Cgc9psxK,BD-Daz-EN,2QpojyCHcD,BD-Daz-EN-DD-App,488a234ad1fa4a11a34bab5b8da5446e,route_daraz_others,2101100088102510,7.005203e+11,...,N,N,Not Available Yet,N,N,N,NaN,Where is my order,NaN,NaN
1403,20220914,Alime-DRZ-CC-BD,l2Cgc9psxK,BD-Daz-EN,0zKpjMUW7x,BD-Daz-EN-HC-Web,d0dd395aeef649e0ac038a39ee81ab46,NaN,2101100093398557,7.000456e+11,...,N,N,Not Available Yet,N,N,Y,NaN,Can I cancel my order?,NaN,NaN
1404,20220914,Alime-DRZ-CC-BD,l2Cgc9psxK,BD-Daz-EN,0zKpjMUW7x,BD-Daz-EN-HC-Web,d0dd395aeef649e0ac038a39ee81ab46,NaN,2101100093398557,7.000456e+11,...,N,N,Not Available Yet,N,N,N,NaN,Can I cancel my order?,NaN,NaN
1405,20220914,Alime-DRZ-CC-BD,l2Cgc9psxK,BD-Daz-EN,0zKpjMUW7x,BD-Daz-EN-HC-Web,d0dd395aeef649e0ac038a39ee81ab46,NaN,2101100093398557,7.000456e+11,...,N,N,Not Available Yet,N,N,N,NaN,Can I cancel my order?,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27498,20220914,Alime-DRZ-CC-BD,l2Cgc9psxK,BD-Daz-EN,0zKpjMUW7x,BD-Daz-EN-HC-Web,3d8ff88929e84742b82b673ff4b9e378,route_daraz_others,2101100093999908,7.005224e+11,...,N,N,Not Available Yet,N,N,N,NaN,Why do I see failed delivery attempt,NaN,NaN
27499,20220914,Alime-DRZ-CC-BD,l2Cgc9psxK,BD-Daz-EN,0zKpjMUW7x,BD-Daz-EN-HC-Web,3d8ff88929e84742b82b673ff4b9e378,NaN,2101100093999908,7.005224e+11,...,N,N,Not Available Yet,N,N,Y,NaN,Why do I see failed delivery attempt,NaN,NaN
27500,20220914,Alime-DRZ-CC-BD,l2Cgc9psxK,BD-Daz-EN,0zKpjMUW7x,BD-Daz-EN-HC-Web,3d8ff88929e84742b82b673ff4b9e378,route_daraz_others,2101100093999908,7.005224e+11,...,N,N,Not Available Yet,N,N,N,NaN,Why do I see failed delivery attempt?,NaN,NaN
28553,20220914,Alime-DRZ-CC-BD,03thkMNkQ3,BD-Daz-LC,zpojXywwrn,BD-Daz-LC-DD-App,b82b92b4a3264846bbf84929c99ac0be,route_daraz_others,2101100093549081,7.005226e+11,...,N,N,Not Available Yet,N,N,N,NaN,অর্ডার কিভাবে ক্যান্সেল করব,NaN,NaN


In [8]:
chat_df_notna = chat_df[chat_df['live agent_session_id'].notna()]

In [ ]:
chat_df_notna[chat_df_notna['create_time_hour'] > 8][]

In [11]:
chat_df_notna[['create_time_hour', 'chat_uuid', 'live agent_session_id']]

,create_time_hour,chat_uuid,live agent_session_id
29,14,7e2e30c9c62f4d1f987867a2c8ebc1d9,ad94dbce03f54a389918e1c867490213
30,14,3e13199dd3184efdb3b8ef40818e688b,ad94dbce03f54a389918e1c867490213
31,14,fcb43f21dc4842159ed69c09308a3cbd,ad94dbce03f54a389918e1c867490213
32,14,186e9ec1be97465399826be967bc4286,ad94dbce03f54a389918e1c867490213
33,14,ce40c219c7bd4aca9725abfeef0c7953,ad94dbce03f54a389918e1c867490213
...,...,...,...
29105,18,3fd2d752fe3149a38f8cd25a99f86c58,ab28c8b2ef1240099c57cbe8873b6e0a
29106,18,9f561a921e2d4ea1ae1c36357acaaf0e,ab28c8b2ef1240099c57cbe8873b6e0a
29107,18,1ffbcc4236be47e4865993ff0d89255d,ab28c8b2ef1240099c57cbe8873b6e0a
29120,18,6e4c36ccfdc34ffa9d4bd47d8ba141c3,2475925f7a1e4a05b0cf0fb3fe8bd38e


In [10]:
chat_df_notna[chat_df_notna['create_time_hour'].isin(hours)][['live agent_session_id', 'chat_uuid', 'create_time_hour']]

,live agent_session_id,chat_uuid,create_time_hour
1073,4061b9cf3aa749b1980d491d933caa6e,9cf2db02cca84bd7b5b68bb202f867e3,1
1074,4061b9cf3aa749b1980d491d933caa6e,5b956d883a3d4bda87288e5b187edc40,1
1075,4061b9cf3aa749b1980d491d933caa6e,c917d479dd0d4385ba5656d493c5b79a,1
1076,4061b9cf3aa749b1980d491d933caa6e,5a0dc28c89664f0d8870796857219937,1
1077,4061b9cf3aa749b1980d491d933caa6e,e96f085e34db451594b66c2536b2bec3,1
1078,4061b9cf3aa749b1980d491d933caa6e,5b242cd722f54f33b7b59884a4cd3292,1
1079,4061b9cf3aa749b1980d491d933caa6e,d14714718c5a429b93dcb45e5f0afc20,1
1080,4061b9cf3aa749b1980d491d933caa6e,e156d6fea64f4f2fa5b3540727ca4434,1
5866,519a7ba3f761496bb8e6a18bdf25e209,f16a88b8af8f4aba82640a23413e6367,4
5867,519a7ba3f761496bb8e6a18bdf25e209,0a02137fe55f425f8b4a8f977344da3d,4


In [279]:
chat_df_non_work_hours = chat_df[chat_df[['create_time_hour', 'live agent_session_id']]['create_time_hour'].isin(hours)]

In [287]:
chat_df_non_work_hours['live agent_session_id']

0        NaN
1        NaN
2        NaN
3        NaN
4        NaN
        ... 
29082    NaN
29083    NaN
29084    NaN
29085    NaN
29086    NaN
Name: live agent_session_id, Length: 4386, dtype: object

In [284]:
chat_df_non_work_hours[chat_df_non_work_hours['live create_time_hour'].notna()]

KeyError: 'live create_time_hour'

In [274]:
chat_df[chat_df['create_time_hour'] == 0]['live agent_session_id'].value_counts()

2ddb0266d81846ad963f333f93299186    6
90e4f18f41be4ca7a066d36cc3d66090    2
dcf3df5544fe43158754e53de0249b10    1
Name: live agent_session_id, dtype: int64

In [198]:
chat_df['Language'].value_counts()

en     21636
bn     13086
und     2615
de       108
hi       104
       ...  
xh         1
fy         1
hr         1
lt         1
ceb        1
Name: Language, Length: 81, dtype: int64

In [203]:
chat_df_en_hc_app = chat_df[chat_df['from_name'] == 'BD-Daz-EN-HC-App']
chat_df_en_hc_web = chat_df[chat_df['from_name'] == 'BD-Daz-EN-HC-Web']

In [204]:
chat_df_en_hc_app.reset_index(inplace=True)

In [205]:
chat_df_en_hc_web.reset_index(inplace=True)

In [206]:
display(chat_df_en_hc_web.head(2))
display(chat_df_en_hc_app.head(2))

,index,Create Date,tenant_name,robot_code,robot_name,from_id,from_name,Session_id,Intention,user_id,...,is_chat_recommend,is_chat_no answer,is_chat_pre/post match,live agent_session_id,Last Query Before Live Agent Session,Live Agent User Query,Live Agent Response,Feedback Type,Unsatisfied Reason,Language
0,12,2022-09-13 00:00:00,Alime-DRZ-CC-BD,l2Cgc9psxK,BD-Daz-EN,0zKpjMUW7x,BD-Daz-EN-HC-Web,1a039da7677b4b8bad77990a58190b48,route_daraz_others,2.101100e+15,...,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,en
1,89,2022-09-13 00:00:00,Alime-DRZ-CC-BD,l2Cgc9psxK,BD-Daz-EN,0zKpjMUW7x,BD-Daz-EN-HC-Web,65e1dddc027a4b279c767e6266e77736,NaN,2.101100e+15,...,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,und


,index,Create Date,tenant_name,robot_code,robot_name,from_id,from_name,Session_id,Intention,user_id,...,is_chat_recommend,is_chat_no answer,is_chat_pre/post match,live agent_session_id,Last Query Before Live Agent Session,Live Agent User Query,Live Agent Response,Feedback Type,Unsatisfied Reason,Language
0,17,2022-09-13 00:00:00,Alime-DRZ-CC-BD,l2Cgc9psxK,BD-Daz-EN,V3EasrtOgs,BD-Daz-EN-HC-App,1b871f26fc144818a3e2c6d725ed775b,route_daraz_others,2.101100e+15,...,N,N,N,NaN,NaN,NaN,NaN,UNSATISFIED,Too many words to read,en
1,18,2022-09-13 00:00:00,Alime-DRZ-CC-BD,l2Cgc9psxK,BD-Daz-EN,V3EasrtOgs,BD-Daz-EN-HC-App,1b871f26fc144818a3e2c6d725ed775b,NaN,2.101100e+15,...,N,N,N,NaN,NaN,NaN,NaN,NaN,NaN,und


In [207]:
display(chat_df_en_hc_web['Intention'].value_counts())
display(chat_df_en_hc_app['Intention'].value_counts())

route_daraz_others       1686
route_daraz_liveagent     294
ordernumber                50
item                        1
Name: Intention, dtype: int64

route_daraz_others       4475
route_daraz_liveagent    1295
ordernumber               142
Name: Intention, dtype: int64

In [208]:
chat_df_en_hc_web_agent = chat_df_en_hc_web[chat_df_en_hc_web['Intention'] == 'route_daraz_liveagent']
chat_df_en_hc_app_agent = chat_df_en_hc_app[chat_df_en_hc_app['Intention'] == 'route_daraz_liveagent']

In [220]:
chat_df_en_hc_web_agent_en = chat_df_en_hc_web_agent[chat_df_en_hc_web_agent['Language'] == 'en']
chat_df_en_hc_web_agent_bn = chat_df_en_hc_web_agent[chat_df_en_hc_web_agent['Language'] == 'bn']


chat_df_en_hc_app_agent_en = chat_df_en_hc_app_agent[chat_df_en_hc_app_agent['Language'] == 'en']
chat_df_en_hc_app_agent_bn = chat_df_en_hc_app_agent[chat_df_en_hc_app_agent['Language'] == 'bn']

In [235]:
chat_df_en_hc_web_agent_en.reset_index(inplace=True)
chat_df_en_hc_web_agent_bn.reset_index(inplace=True)

chat_df_en_hc_app_agent_en.reset_index(inplace=True)
# chat_df_en_hc_app_agent_bn.reset_index(inplace=True)

In [236]:
chat_df_en_hc_app_agent_bn['query']

0          Kono phone number ache jogajog korar jonno
1                                  live a kotha bolbo
2         Ami akjon agent er shathe kotha bolte chai.
3                         Agent er shathe kotha bolbo
4                                   kivabe agent pabo
5                                         Agent lagbe
6                          Agent er sathe kotha bolbo
7                Customer care er sate kota bolte cai
8                                             Jogajog
9         Customer manager er Sathe Kotha bolted chai
10                   Agent ar sathe kotha bolte cassi
11    Daraz customer care agent er shathe kotha bolbo
12                   Agent ar sathe kotha bolte chai?
13        Ami akjon agent er shathe kotha bolte chai.
14                                Aponar help  dorkar
15                     Agent er sate kotha  bolte cai
16                Agent er shathe kotha bolte chacchi
17                                        Agent lagbe
18                          

In [261]:
chat_df_en_hc_web_agent_bn['query'].value_counts()[1:50]

Ami kotha bolte cai                                                 1
agent er sathe kota kobo                                            1
Jogajog                                                             1
Apni kindly karor contact number din jar sathe jogajog kora jabe    1
Customer manager er sathe kotha bolte cai                           1
Name: query, dtype: int64

In [226]:

display(chat_df_en_hc_web_agent_en['query'].count())
display(chat_df_en_hc_web_agent_bn['query'].count())


274

6

In [227]:
display(chat_df_en_hc_app_agent_en['query'].count())
display(chat_df_en_hc_app_agent_bn['query'].count())


1229

21

In [230]:
chat_df_en_hc_app_agent_en['query']

53                 Live agent
59       I want to chat agent
60              service agent
61                     Agent"
66                 Live agent
                 ...         
14970           Contact agent
14974              live agent
15009                   Agent
15014                   Agent
15034                   Agent
Name: query, Length: 1229, dtype: object

In [ ]:
chat_df_en_hc_web_agent_bn

In [168]:
from nltk.stem import PorterStemmer
from nltk.tokenize import WhitespaceTokenizer


from nltk.corpus import words

# nltk.download('words')
  
correct_words = words.words()

In [169]:
sentence = "Customerssupport"

In [173]:
def check_spacing(term):
    possibles = []
    for i in range(1, len(term) - 1):
        l, r = term[:i], term[i:]

        if l in correct_words and r in correct_words:
            possibles.append((l, r))

    # probs_check is a theoretical function that returns a numeric value
    # which determines how likely each pair of words is to be what you want
    possibles = sorted(possibles, reversed=True)

    return ' '.join(possibles[0])

In [ ]:
check_spacing()

In [170]:
tk = WhitespaceTokenizer()

In [171]:
tk.tokenize(sentence)

['Customerssupport']

In [160]:
ps = PorterStemmer()

In [165]:
sentence = "Customerssupport"

In [166]:
words = word_tokenize(sentence)

In [167]:
words

['Customerssupport']

In [134]:

# importing the nltk suite 
import nltk
  
# importing jaccard distance
# and ngrams from nltk.util
from nltk.metrics.distance import jaccard_distance
from nltk.util import ngrams

In [ ]:
from nltk.corpus import words
correct_words = words.words()

incorrect_words=['agent✅✅✅', 'ageny', 'agggntt', 'yagent']
  
# loop for finding correct spellings
# based on edit distance and
# printing the correct words
for word in incorrect_words:
    temp = [(edit_distance(word, w),w) for w in correct_words if w[0]==word[0]]
    print(sorted(temp, key = lambda val:val[0])[0][1])

In [23]:
len(chat_df_en_hc_app_agent['query'])

1295

In [129]:
chat_df_en_hc_app_agent['query'].to_clipboard()

In [135]:
nltk.download('words')
from nltk.corpus import words
  
  
correct_words = words.words()

[nltk_data] Downloading package words to
[nltk_data]     C:\Users\Daraz\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [144]:
incorrect_words=['Agennnt', 'Agentttt', 'aagnt']
  
# loop for finding correct spellings
# based on jaccard distance
# and printing the correct word
for word in incorrect_words:
    temp = [(jaccard_distance(set(ngrams(word, 2)),
                              set(ngrams(w, 2))),w)
            for w in correct_words if w[0]==word[0]]
    print(sorted(temp, key = lambda val:val[0])[0][1])

Agena
Agena
agent


In [175]:
from __future__ import division
from collections import Counter
import re, nltk

WORDS = nltk.corpus.brown.words()
COUNTS = Counter(WORDS)

In [178]:
def pdist(counter):
    "Make a probability distribution, given evidence from a Counter."
    N = sum(counter.values())
    return lambda x: counter[x]/N

P = pdist(COUNTS)

def Pwords(words):
    "Probability of words, assuming each word is independent of others."
    return product(P(w) for w in words)

def product(nums):
    "Multiply the numbers together.  (Like `sum`, but with multiplication.)"
    result = 1
    for x in nums:
        result *= x
    return result

def splits(text, start=0, L=20):
    "Return a list of all (first, rest) pairs; start <= len(first) <= L."
    return [(text[:i], text[i:]) 
            for i in range(start, min(len(text), L)+1)]

def segment(text):
    "Return a list of words that is the most probable segmentation of text."
    if not text: 
        return []
    else:
        candidates = ([first] + segment(rest) 
                      for (first, rest) in splits(text, 1))
        return max(candidates, key=Pwords)

print(segment('Customersupport'))

['Customer', 'support']


In [174]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
sentence="Customersupport"
words = word_tokenize(sentence)
ps = PorterStemmer()
for w in words:
	rootWord=ps.stem(w)
	print(rootWord)

customersupport


In [157]:
# importing the nltk suite
import nltk

# importing edit distance
from nltk.metrics.distance import edit_distance

from nltk.corpus import words
correct_words = words.words()

incorrect_words=['agent✅✅✅', 'ageny', 'agggntt', 'yagent']
  
# loop for finding correct spellings
# based on edit distance and
# printing the correct words
for word in incorrect_words:
    temp = [(edit_distance(word, w),w) for w in correct_words if w[0]==word[0]]
    print(sorted(temp, key = lambda val:val[0])[0][1])

agent
agen
against
yamen


In [158]:
mystring="chat with Customer agenttts"
list_of_words=["agent"]

word_counts = {}

from nltk.stem.snowball import EnglishStemmer
stemmer = EnglishStemmer()

for target in list_of_words:
    word_counts[target] = 0

    for word in mystring.split(' '):

        # Stem the word and compare it to the stem of the target
        stem = stemmer.stem(word)        
        if stem == stemmer.stem(target):
            word_counts[target] += 1

print(word_counts)

{'agent': 0}


In [92]:
chat_df_en_hc_app_agent_false1 = chat_df_en_hc_app_agent[chat_df_en_hc_app_agent['query'].str.match(r'[aA]+[gG]+') == False]

In [111]:
chat_df_en_hc_app_agent_false1['query'].to_clipboard()

In [128]:
chat_df_en_hc_app_agent_false1[chat_df_en_hc_app_agent_false1['query'].str.match(r'.[agent].') == False]['query'].to_clipboard()

In [36]:
if re.search('agent', 'agent', re.IGNORECASE):
    print('hello!')

In [24]:
len(chat_df_en_hc_web_agent['query'])

294

In [25]:
chat_df_en_hc_web_agent['query'].to_clipboard()

In [6]:
DDA_0619['Feedback Type'].value_counts()

UNSATISFIED    516
SATISFIED      460
Name: Feedback Type, dtype: int64

In [4]:
DDA_0619.isna().sum()

Create Date                                 0
tenant_name                                 0
robot_code                                  0
robot_name                                  0
from_id                                     0
from_name                                   0
Session_id                                  0
Intention                                8385
user_id                                     0
Seller id (X-space)                         0
user_nick                                   0
create_time                                 0
chat_uuid                                   0
Enter Type                              16379
query                                       0
Question Type                               0
sopname                                 21229
answer_type                               140
SOP_Solution_id                         24854
SOP_Solution_Name                       24494
Standard Knowledge Title                 7845
Knowledge_id                      

In [6]:
DDA_0619['robot_name'].value_counts()

BD-Daz-EN    22306
BD-Daz-LC     8468
Name: robot_name, dtype: int64

In [7]:
DDA_0619_EN = DDA_0619[DDA_0619['robot_name'] == 'BD-Daz-EN']
DDA_0619_LC = DDA_0619[DDA_0619['robot_name'] == 'BD-Daz-LC']

In [8]:
DDA_0619_EN.shape

(22306, 42)

In [9]:
DDA_0619_LC.shape

(8468, 42)

In [10]:
DDA_0619.columns

Index(['Create Date', 'tenant_name', 'robot_code', 'robot_name', 'from_id',
       'from_name', 'Session_id', 'Intention', 'user_id',
       'Seller id (X-space)', 'user_nick', 'create_time', 'chat_uuid',
       'Enter Type', 'query', 'Question Type', 'sopname', 'answer_type',
       'SOP_Solution_id', 'SOP_Solution_Name', 'Standard Knowledge Title',
       'Knowledge_id', 'knowledge_cate_1_name', 'knowledge_cate_2_name',
       'knowledge_cate_3_name', 'is_session_unsatisfied',
       'is_session_last_no answer', 'call_agent_by_click_shortcut',
       'is_session_last_recommend no click', 'call_agent_by_intention',
       'is_session_direct_liveagent', 'Contacted live agent on same day',
       'Feedback Card Response', 'is_chat_recommend', 'is_chat_no answer',
       'is_chat_pre/post match', 'live agent_session_id',
       'Last Query Before Live Agent Session', 'Live Agent User Query',
       'Live Agent Response', 'Feedback Type', 'Unsatisfied Reason'],
      dtype='object')

##### Total number of EN chats

In [11]:
#axis = 1 vertical
#axis = 0 horizontal
# DDA_LC_APP_CONCAT_25_28.drop(columns='index', axis=1, inplace=True)

DDA_0619_EN['is_chat_no answer'].value_counts()

N    22306
Name: is_chat_no answer, dtype: int64

In [12]:
DDA_0619_LC['is_chat_no answer'].value_counts()

N    8328
Y     140
Name: is_chat_no answer, dtype: int64

In [13]:
DDA_0619_EN['Session_id'].value_counts()

8a4045fc031c4627841ff6309f5ab816    152
dc2d5c1aff18435f9d2fa107befc5bbd     46
828611be4a064ca8a39812f6ca8d6ceb     40
a9019500c87949e7a3bfce9b7d7ab792     40
fac94dad072d45cda7253f7d6cb4f1e9     38
                                   ... 
2791ac93054f4c4dbb6df865192cc9fb      1
23f0d7a0b6794aa6b5fa7ba047d40493      1
23712c747b274a60a418f51cc3066bf2      1
ca1291988bc2407db3e024aaa107ee85      1
45dd62b0bf9d449b95f6601592db7a08      1
Name: Session_id, Length: 5505, dtype: int64

In [14]:
DDA_0619_EN_drop_duplicates = DDA_0619_EN.drop_duplicates(subset=['Session_id'], keep='last')
DDA_0619_LC_drop_duplicates = DDA_0619_LC.drop_duplicates(subset=['Session_id'], keep='last')

In [15]:
display(DDA_0619_EN_drop_duplicates.shape)
display(DDA_0619_LC_drop_duplicates.shape)

(5505, 42)

(2885, 42)

In [16]:
DDA_0619_EN_drop_duplicates.reset_index(inplace=True)
DDA_0619_LC_drop_duplicates.reset_index(inplace=True)

In [17]:
DDA_0619_EN_drop_duplicates['Standard Knowledge Title'].dropna(inplace=True)
DDA_0619_LC_drop_duplicates['Standard Knowledge Title'].dropna(inplace=True)

C:\Users\Daraz\AppData\Local\Temp/ipykernel_35564/509692275.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DDA_0619_EN_drop_duplicates['Standard Knowledge Title'].dropna(inplace=True)
C:\Users\Daraz\AppData\Local\Temp/ipykernel_35564/509692275.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DDA_0619_LC_drop_duplicates['Standard Knowledge Title'].dropna(inplace=True)


In [18]:
DDA_0619_EN_drop_duplicates['Standard Knowledge Title'].value_counts().to_clipboard()
# DDA_0619_LC_drop_duplicates['Standard Knowledge Title'].value_counts().to_clipboard()

In [19]:
DDA_0619_EN_drop_nan = DDA_0619_EN_drop_duplicates[DDA_0619_EN_drop_duplicates['Live Agent User Query'].notna()]
DDA_0619_LC_drop_nan = DDA_0619_LC_drop_duplicates[DDA_0619_LC_drop_duplicates['Live Agent User Query'].notna()]

In [20]:
DDA_0619_EN_drop_nan.reset_index(inplace=True)
DDA_0619_LC_drop_nan.reset_index(inplace=True)

In [21]:
DDA_0619_EN_drop_nan.drop(labels=['level_0', 'index'], axis = 1, inplace=True)
DDA_0619_LC_drop_nan.drop(labels=['level_0', 'index'], axis = 1, inplace=True)

C:\Users\Daraz\AppData\Local\Temp/ipykernel_35564/285115638.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DDA_0619_EN_drop_nan.drop(labels=['level_0', 'index'], axis = 1, inplace=True)
C:\Users\Daraz\AppData\Local\Temp/ipykernel_35564/285115638.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  DDA_0619_LC_drop_nan.drop(labels=['level_0', 'index'], axis = 1, inplace=True)


In [47]:
DDA_0619_LC_drop_nan['Standard Knowledge Title'].value_counts().to_clipboard()
# DDA_0619_EN_drop_nan['Standard Knowledge Title'].value_counts().to_clipboard()


In [39]:
kt = 'আমার অর্ডার আসতে দেরি হচ্ছে কেন ?'
# n = 0
n = np.random.randint(20)

DDA_EN_KT = DDA_0619_LC_drop_nan[DDA_0619_LC_drop_nan['Standard Knowledge Title'] == kt]
DDA_EN_KT.reset_index(inplace=True)


print('Standard Knowledge Title')

display(DDA_EN_KT['Standard Knowledge Title'][n])

print('Last Query Before Live Agent Session')

display(DDA_EN_KT['Last Query Before Live Agent Session'][n])

print('Live Agent User Query')

display(DDA_EN_KT['Live Agent User Query'][n])

print('Live Agent Response')

display(DDA_EN_KT['Live Agent Response'][n])

Standard Knowledge Title


'আমার অর্ডার আসতে দেরি হচ্ছে কেন ?'

Last Query Before Live Agent Session


'আমার অর্ডার আসতে দেরি হচ্ছে কেন'

Live Agent User Query


'Amar order akhono asheni keno'

Live Agent Response


'Onugroho kore apnar order number ti share korben . Ami order er details check kore janacchi.  |||Apni ki amader shathe achen?  |||Ami dukkhito sir bishoy tir jonno.|||Hi there! Welcome to Daraz Bangladesh. Hope you’re doing fine by the grace of the Almighty. How can I assist you?'

In [40]:
DDA_EN_KT_selected = DDA_EN_KT[['query', 'Standard Knowledge Title', 'Live Agent User Query', 'Live Agent Response']]

In [41]:
DDA_EN_KT_selected[DDA_EN_KT_selected['Standard Knowledge Title'] == 'Need help']

,query,Standard Knowledge Title,Live Agent User Query,Live Agent Response


### Conclusion

15/25 -> Different Questions
6/25 -> Repeatitive Questions
1/25 -> Red marked



In [170]:
df_test = DDA_0619_LC_drop_nan[DDA_0619_LC_drop_nan['Standard Knowledge Title'] == 'রিটার্ন সংক্রান্ত প্রশ্ন'][['query', 'Standard Knowledge Title', 'Live Agent User Query', 'Live Agent Response']]

In [171]:
df_test

,query,Standard Knowledge Title,Live Agent User Query,Live Agent Response
84,রিটার্ন সংক্রান্ত প্রশ্ন,রিটার্ন সংক্রান্ত প্রশ্ন,hmm|||Amar Ekta percel.. Return korbi|||<img s...,Hello! Warm Welcome to Daraz live chat suppor...
229,রিটার্ন সংক্রান্ত প্রশ্ন,রিটার্ন সংক্রান্ত প্রশ্ন,Hi|||Ok|||Ata amar home ar akane asbe naki bah...,Apni jodi aaj amar service ti pocchondo kore t...
313,রিটার্ন সংক্রান্ত প্রশ্ন,রিটার্ন সংক্রান্ত প্রশ্ন,চেক করব|||<img src='https://alime-customer-upl...,স্যার দেখতে পাচ্ছি আপনার রিফান্ড টি প্রসেসিং এ...
392,রিটার্ন সংক্রান্ত প্রশ্ন,রিটার্ন সংক্রান্ত প্রশ্ন,রিটার্ন প্রোসেস কি শেষ হয়েছে?|||আমার রিটার্ন প...,স্যার ভাউচার কোডটি কপি পেস্ট করে আমার সাথে শেয়...


In [172]:
df_test.reset_index(inplace=True)

In [173]:
df_test.drop(labels='index', axis = 1, inplace = True)

In [174]:
df_test.shape

(4, 4)

In [167]:
n = np.random.randint(df_test.shape[0] - 1)
df_test.iloc[[n]]

,query,Standard Knowledge Title,Live Agent User Query,Live Agent Response
2,শিপিং এবং ডেলিভেরি সক্রান্ত প্রশ্ন,শিপিং এবং ডেলিভেরি সক্রান্ত প্রশ্ন,<img src='https://alime-customer-upload-lazada...,স্যার অর্ডার নম্বর টি এখানে শেয়ার করুন|||আন্তর...


In [168]:
df_test.iloc[[n]].to_clipboard(excel=True, sep=None, index=False, header=None)

In [169]:
df_test.to_clipboard(excel=True, sep=None, index=False, header=None)

In [129]:
lc_df = pd.read_excel(r'DDA 17th July (1).xlsx', sheet_name='LC')

In [128]:
en_df  = pd.read_excel(r'DDA 17th July (1).xlsx', sheet_name='EN')

In [135]:
en_df['Remark'].value_counts().to_clipboard()

In [134]:
lc_df['Remark'].value_counts().sum()

47

In [132]:
en_df

,Query,Standard Knowledge Title,Live Agent User Query,Live Agent Response,Remark
0,How can I track my order',How can I track my order',I had ordered a piano from chittagong in 7th j...,"""We can see the issue has been forwarded to ou...",order delay
1,How can I track my order',How can I track my order',Naki be Nebo Ata at pabo na|||Amar ai order ti...,"Please share your query in details with us, so...",order delay
2,Where is my order',How can I track my order',DEX-BDN-CB-0000693055|||Please give me the upd...,Kindly allow me some time. |||Good day! Welcom...,order delay
3,How can I track my order',How can I track my order',"""When I'm gonna get my product?|||ok thanks|||...",Hello! Welcome to Daraz live chat support. 😊 H...,order delay
4,How can I track my order',How can I track my order',"""629074632994006|||Sure |||Amar ei order ta as...","Sir ami ki kichuta somoy pete pari, check kora...",order delay
5,How can I track my order',How can I track my order',Dcoin|||আর চালু হবে না? |||😭|||আর কোন আপডেট হব...,পেয়ে থাকবেন|||না স্যার|||Hello sir|||মিশন কমপ্...,different question to agent
6,How can I track my order',How can I track my order',":ok thanks, duruto delivery diben pls|||amr or...","Sir, Bishoyti amader concern team er kache jan...",delivery attempt failed
7,How can I track my order',How can I track my order',Paperfly basar samne ashe na |||Eigula ki dhor...,sir asha korchi apnar shathe punoray delivery ...,failed delivery
8,How can I track my order',How can I track my order',订单号:628373899747135|||Then complain korar por ...,订单号:628373899747135|||Then complain korar por ...,refund delay
9,How can I track my order',How can I track my order',This is the third time am contacting with dara...,"Hello Mam😊|||Sure, Mam 😊|||Good Day!! I hope y...",order delay\nwants to know rider number
